In [2]:
# Install Flask and SQLite support (required for running the API)
!pip install flask flask_sqlalchemy


In [3]:
from google.colab import files

# Upload the CSV file manually
uploaded = files.upload()

# Confirm the filename
csv_filename = list(uploaded.keys())[0]
print(f"✅ CSV file uploaded: {csv_filename}")


Saving TestData.csv to TestData (1).csv
✅ CSV file uploaded: TestData (1).csv


In [4]:
# Function to parse CSV without using standard libraries
def parse_csv(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()  # Read all lines

    data = []
    for line in lines[1:]:  # Skip header
        parts = line.strip().split(",")  # Manually split by commas
        if len(parts) == 3:  # Ensure correct format
            first_name, last_name, score = parts
            full_name = f"{first_name.strip()} {last_name.strip()}"  # Combine first and last name
            data.append((full_name, int(score.strip())))  # Store as tuple

    return data

# Parse the uploaded CSV
parsed_data = parse_csv(csv_filename)
print("✅ CSV Data Parsed Successfully!")
print(parsed_data[:5])  # Show first 5 entries for verification


✅ CSV Data Parsed Successfully!
[('Dee Moore', 56), ('Sipho Lolo', 78), ('Noosrat Hoosain', 64), ('George Of The Jungle', 78)]


In [5]:
import sqlite3

# Initialize SQLite database and create table
def init_db():
    conn = sqlite3.connect("scores.db")
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS scores (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            score INTEGER NOT NULL
        )
    ''')
    conn.commit()
    conn.close()

# Function to insert parsed CSV data into SQLite
def insert_data(data):
    conn = sqlite3.connect("scores.db")
    cursor = conn.cursor()
    cursor.executemany("INSERT INTO scores (name, score) VALUES (?, ?)", data)
    conn.commit()
    conn.close()

# Run database setup
init_db()
insert_data(parsed_data)

print("✅ Data inserted into SQLite successfully!")


✅ Data inserted into SQLite successfully!


In [6]:
# Function to get top scorers from the database
def get_top_scorers():
    conn = sqlite3.connect("scores.db")
    cursor = conn.cursor()
    cursor.execute("SELECT name, score FROM scores ORDER BY score DESC")
    results = cursor.fetchall()
    conn.close()

    if not results:
        return None

    top_score = results[0][1]  # Get highest score
    top_scorers = sorted([name for name, score in results if score == top_score])  # Sort names alphabetically

    return top_scorers, top_score

# Display results
top_scorers, top_score = get_top_scorers()
if top_scorers:
    print("🏆 Top Scorers:", ", ".join(top_scorers))
    print("⭐ Score:", top_score)
else:
    print("⚠️ No scores found in the database.")


🏆 Top Scorers: George Of The Jungle, Sipho Lolo
⭐ Score: 78


In [7]:
from flask import Flask, request, jsonify

app = Flask(__name__)

# API: Add a new score
@app.route('/scores', methods=['POST'])
def add_score():
    data = request.get_json()
    name = data.get("name")
    score = data.get("score")

    if not name or score is None:
        return jsonify({"error": "Name and score are required."}), 400

    conn = sqlite3.connect("scores.db")
    cursor = conn.cursor()
    cursor.execute("INSERT INTO scores (name, score) VALUES (?, ?)", (name, score))
    conn.commit()
    conn.close()

    return jsonify({"message": "Score added successfully."}), 201

# API: Get a score by name
@app.route('/scores/<name>', methods=['GET'])
def get_score(name):
    conn = sqlite3.connect("scores.db")
    cursor = conn.cursor()
    cursor.execute("SELECT score FROM scores WHERE name = ?", (name,))
    result = cursor.fetchone()
    conn.close()

    if result:
        return jsonify({"name": name, "score": result[0]})
    else:
        return jsonify({"error": "Score not found."}), 404

# API: Get top scorers
@app.route('/top-scorers', methods=['GET'])
def get_top_scorers_api():
    result = get_top_scorers()
    if result is None:
        return jsonify({"error": "No scores available."}), 404
    top_scorers, top_score = result
    return jsonify({"top_scorers": top_scorers, "score": top_score})

# Start the Flask server
app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
